In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
!pip install -U transformers
!pip install datasets
!pip install fsspec==2021.5.0
!pip install jiwer==2.2.0

In [3]:
import os
import torch
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric,Dataset,concatenate_datasets,set_caching_enabled, ClassLabel
import pandas as pd

import random
from IPython.display import display, HTML

import json
from transformers import Wav2Vec2CTCTokenizer,Wav2Vec2ForCTC,Wav2Vec2Processor,Trainer,TrainingArguments,Wav2Vec2FeatureExtractor

import re
set_caching_enabled(False)

import soundfile as sf
import torchaudio

import IPython.display as ipd

import numpy as np
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

from tqdm import tqdm
import torch

# Set environment variables
os.environ['WANDB_DISABLED '] = 'True'

import warnings
warnings.filterwarnings('ignore')

<ipython-input-3-2b4ebb15e84c>:14: FutureWarning: set_caching_enabled is deprecated and will be removed in the next major version of datasets. Use datasets.enable_caching() or datasets.disable_caching() instead. This function will be removed in a future version of datasets.
  set_caching_enabled(False)


In [4]:
import logging
import transformers
transformers.logging.get_verbosity = lambda: logging.NOTSET

In [5]:
transformers.logging.get_verbosity()

0

In [6]:
import datasets
datasets.logging.get_verbosity = lambda: logging.NOTSET

In [56]:
df =pd.read_csv("/content/drive/MyDrive/PROJET_DATA@354_NLP/train.csv")
df = df.head(1000)

#Add audio path and renaming columns
df["path"] = "/content/drive/MyDrive/PROJET_DATA@354_NLP/clips/clips/"+df["ID"]+".mp3"
df.rename(columns = {'transcription':'sentence'}, inplace = True)

train,test = train_test_split(df, test_size=0.05, random_state=42)
common_voice_train_1 = Dataset.from_pandas(train[500:])
common_voice_train_2 = Dataset.from_pandas(train[0:500])

common_voice_test = Dataset.from_pandas(test)

In [8]:
df.head()

,ID,sentence,path
0,04f9284cdbd8d747b569b07394b5815134e42bd242ad5f...,Albert Sarraut,/content/drive/MyDrive/PROJET_DATA@354_NLP/cli...
1,61c1af5b4f1788b528d2ef73ef1b1360adb22ea2823b6c...,Grande Mosquee de Dakar,/content/drive/MyDrive/PROJET_DATA@354_NLP/cli...
2,78f1bee2e7b0c7ef76228dbe0c7a3b53f6a7531f1f00d3...,SAMU Municipale Grand-Yoff,/content/drive/MyDrive/PROJET_DATA@354_NLP/cli...
3,3eb6f508bc1ac50b1e68ddea7460fc385f79cee860e463...,Radio Futurs Media (RFM),/content/drive/MyDrive/PROJET_DATA@354_NLP/cli...
4,009ec9a9068f3fe550e6cb9c809ebbfa4ba4c478703c1d...,Lycée Thierno Seydou Nourou Tall,/content/drive/MyDrive/PROJET_DATA@354_NLP/cli...


In [9]:
df['path'][0]

'/content/drive/MyDrive/PROJET_DATA@354_NLP/clips/clips/04f9284cdbd8d747b569b07394b5815134e42bd242ad5f49f17658d521ac2455da87f7d1159196f16637e5639940ccb0aee8df0682f6054b2b9cbd806b455002.mp3'

In [10]:
sum(df.sentence.value_counts()>1)

295

In [11]:
train.shape,test.shape

((950, 3), (50, 3))

In [57]:
common_voice_train_1 = common_voice_train_1.remove_columns([ "ID"])
common_voice_train_2 = common_voice_train_2.remove_columns([ "ID"])

common_voice_test = common_voice_test.remove_columns([ "ID"])

In [13]:
common_voice_train_2.shape

(500, 3)

In [14]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [15]:
show_random_elements(common_voice_train_1.remove_columns(["path"]), num_examples=7)

,sentence,__index_level_0__
0,Elton Ouest Foire,262
1,Grand Yoff Cinéma Christa,877
2,Poste Courant,856
3,Sandaga laa bëgg dem,71
4,Pharmacie Bousso dramé,915
5,Sapeur Pompier de Rufisque,180
6,Blaise Sengor,760


In [16]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [58]:
common_voice_train_1 = common_voice_train_1.map(remove_special_characters)
common_voice_train_2 = common_voice_train_2.map(remove_special_characters)

common_voice_test = common_voice_test.map(remove_special_characters)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [18]:
show_random_elements(common_voice_train_1.remove_columns(["path"]))

,sentence,__index_level_0__
0,stade demba diop,858
1,police golf sud,45
2,ban oto mooy dem saveurs d'asie,791
3,imprimerie tandjan,932
4,pharmacie baye niasse,147
5,pont yarakh,150
6,hopital ouakam,345
7,marché bou béss,438
8,lpa,374
9,terminus lac rose,16


In [19]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [20]:
vocab_train_2 = common_voice_train_2.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train_2.column_names)

vocab_train_1 = common_voice_train_1.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train_1.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [21]:
vocab_list = list(set(vocab_train_2["vocab"][0]) |set(vocab_train_1["vocab"][0]) | set(vocab_test["vocab"][0]))

In [22]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 0,
 'â': 1,
 'z': 2,
 's': 3,
 'w': 4,
 '(': 5,
 'v': 6,
 'l': 7,
 'u': 8,
 'ë': 9,
 'r': 10,
 'a': 11,
 'p': 12,
 'm': 13,
 'd': 14,
 'i': 15,
 'x': 16,
 'o': 17,
 'h': 18,
 'œ': 19,
 ')': 20,
 '’': 21,
 "'": 22,
 'k': 23,
 'j': 24,
 'ô': 25,
 'c': 26,
 'e': 27,
 't': 28,
 'è': 29,
 'y': 30,
 'q': 31,
 'f': 32,
 'g': 33,
 'n': 34,
 'b': 35,
 'î': 36,
 'é': 37}

In [23]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [24]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

40

In [141]:
with open('/content/drive/MyDrive/PROJET_DATA@354_NLP/vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [25]:
tokenizer = Wav2Vec2CTCTokenizer('/content/drive/MyDrive/PROJET_DATA@354_NLP/vocab.json', unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [26]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [27]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [28]:
processor.save_pretrained("/content/drive/MyDrive/PROJET_DATA@354_NLP/wav2vec2-large-xlsr-WOLOF")

In [29]:
common_voice_train_1[0]

{'sentence': 'hopital abass ndao ',
 'path': '/content/drive/MyDrive/PROJET_DATA@354_NLP/clips/clips/0abd1990d6181970a12c498f8fb030f04fc3535ba51a740d299067b63b614cb8dd93467c1bdb5ab91c089125234c8f3f656ddee90c8843afbb4a095fef607fc6.mp3',
 '__index_level_0__': 116}

In [59]:
import librosa

def speech_file_to_array_fn(batch):
    
    speech_array, sampling_rate = librosa.load(batch["path"], sr=None)
    
    
    speech_array_resampled = librosa.resample(speech_array, sampling_rate, 16000)
    sampling_rate_resampled = 16000
    
    
    batch["speech"] = speech_array_resampled
    batch["sampling_rate"] = sampling_rate_resampled
    batch["target_text"] = batch["sentence"]
    
    return batch
    
def speech_file_to_array_fn_test(batch):
    
    speech_array, sampling_rate = librosa.load(batch["path"], sr=None)
    
    
    speech_array_resampled = librosa.resample(speech_array, sampling_rate, 16000)
    sampling_rate_resampled = 16000
    
    
    batch["speech"] = speech_array_resampled
    batch["sampling_rate"] = sampling_rate_resampled
    
    return batch


In [ ]:
help(sf)

In [32]:
common_voice_test.column_names

['sentence', 'path', '__index_level_0__']

In [60]:
common_voice_train_1 = common_voice_train_1.map(speech_file_to_array_fn, remove_columns=common_voice_train_1.column_names,num_proc=4)
common_voice_train_2 = common_voice_train_2.map(speech_file_to_array_fn, remove_columns=common_voice_train_2.column_names,num_proc=4)

common_voice_test = common_voice_test.map(speech_file_to_array_fn, remove_columns=common_voice_test.column_names,num_proc=4)

Map (num_proc=4):   0%|          | 0/50 [00:00<?, ? examples/s]

In [34]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train_1)-1)

ipd.Audio(data=np.asarray(common_voice_train_1[rand_int]["speech"]), autoplay=True, rate=16000)

In [35]:
rand_int = random.randint(0, len(common_voice_train_1)-1)

print("Target text:", common_voice_train_1[rand_int]["target_text"])
print("Input array shape:", np.asarray(common_voice_train_1[rand_int]["speech"]).shape)
print("Sampling rate:", common_voice_train_1[rand_int]["sampling_rate"])

Target text: sgbs grand dakar 
Input array shape: (46464,)
Sampling rate: 16000


In [36]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [37]:
def prepare_dataset_test(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], padding=True,sampling_rate=batch["sampling_rate"][0]).input_values
    
    return batch

In [38]:
common_voice_train_1.shape,common_voice_test.shape

((450, 3), (50, 3))

In [39]:
common_voice_train_1 = common_voice_train_1.map(prepare_dataset, remove_columns=common_voice_train_1.column_names, batch_size=8, num_proc=4, batched=True)
common_voice_train_2 = common_voice_train_2.map(prepare_dataset, remove_columns=common_voice_train_2.column_names, batch_size=8, num_proc=4, batched=True)

Map (num_proc=4):   0%|          | 0/450 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/500 [00:00<?, ? examples/s]

In [61]:
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, batch_size=8, num_proc=4, batched=True)

Map (num_proc=4):   0%|          | 0/50 [00:00<?, ? examples/s]

In [41]:
common_voice_train = concatenate_datasets([common_voice_train_1, common_voice_train_2])

In [157]:
import pickle
with open("/content/drive/MyDrive/PROJET_DATA@354_NLP/common_voice_train.pkl", "wb") as f:
  pickle.dump(common_voice_train, f)
  f.close()

In [62]:
import pickle
with open("/content/drive/MyDrive/PROJET_DATA@354_NLP/common_voice_test.pkl", "wb") as f:
  pickle.dump(common_voice_test, f)
  f.close()

In [20]:
import pickle
with open("/content/drive/MyDrive/PROJET_DATA@354_NLP/common_voice_train.pkl", "rb") as f:
  common_voice_train = pickle.load(f)

In [43]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )


        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [63]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [45]:
wer_metric = load_metric("wer")

In [46]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [47]:
len(processor.tokenizer)

40

In [64]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--facebook--wav2vec2-large-xlsr-53/snapshots/c3f9d884181a224a6ac87bf8885c84d1cff3384f/config.json
Model config Wav2Vec2Config {
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_

In [65]:
model.freeze_feature_extractor()

In [66]:
training_args = TrainingArguments(
  output_dir="/content/drive/MyDrive/PROJET_DATA@354_NLP/wav2vec2-large-xlsr-WOLOF",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=40,
  fp16=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=3e-4,
  warmup_steps=1000,
  save_total_limit=2,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [67]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

Using cuda_amp half precision backend


In [52]:
os.environ['WANDB_MODE'] = 'dryrun'
os.environ['WANDB_DISABLED '] = 'True'

In [68]:
common_voice_train.shape

(950, 2)

In [69]:
trainer.train()#

***** Running training *****
  Num examples = 950
  Num Epochs = 40
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 1200
  Number of trainable parameters = 311269544


Step,Training Loss,Validation Loss,Wer
500,7.585100,2.535081,1.000000
1000,0.849500,0.339069,0.980000


***** Running Evaluation *****
  Num examples = 50
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/PROJET_DATA@354_NLP/wav2vec2-large-xlsr-WOLOF/checkpoint-500
Configuration saved in /content/drive/MyDrive/PROJET_DATA@354_NLP/wav2vec2-large-xlsr-WOLOF/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/PROJET_DATA@354_NLP/wav2vec2-large-xlsr-WOLOF/checkpoint-500/pytorch_model.bin
Feature extractor saved in /content/drive/MyDrive/PROJET_DATA@354_NLP/wav2vec2-large-xlsr-WOLOF/checkpoint-500/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 50
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/PROJET_DATA@354_NLP/wav2vec2-large-xlsr-WOLOF/checkpoint-1000
Configuration saved in /content/drive/MyDrive/PROJET_DATA@354_NLP/wav2vec2-large-xlsr-WOLOF/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/PROJET_DATA@354_NLP/wav2vec2-large-xlsr-WOLOF/checkpoint-1000/pytorch_model.bin
Feature extract

TrainOutput(global_step=1200, training_loss=3.550320472717285, metrics={'train_runtime': 2633.746, 'train_samples_per_second': 14.428, 'train_steps_per_second': 0.456, 'total_flos': 3.793605815815004e+18, 'train_loss': 3.550320472717285, 'epoch': 40.0})

In [70]:
model.save_pretrained("/content/drive/MyDrive/PROJET_DATA@354_NLP/wav2vec2-large-xlsr-WOLOF")
processor.save_pretrained("/content/drive/MyDrive/PROJET_DATA@354_NLP/wav2vec2-large-xlsr-WOLOF")

Configuration saved in /content/drive/MyDrive/PROJET_DATA@354_NLP/wav2vec2-large-xlsr-WOLOF/config.json
Model weights saved in /content/drive/MyDrive/PROJET_DATA@354_NLP/wav2vec2-large-xlsr-WOLOF/pytorch_model.bin
Feature extractor saved in /content/drive/MyDrive/PROJET_DATA@354_NLP/wav2vec2-large-xlsr-WOLOF/preprocessor_config.json
tokenizer config file saved in /content/drive/MyDrive/PROJET_DATA@354_NLP/wav2vec2-large-xlsr-WOLOF/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/PROJET_DATA@354_NLP/wav2vec2-large-xlsr-WOLOF/special_tokens_map.json


In [71]:
val =pd.read_csv("/content/drive/MyDrive/PROJET_DATA@354_NLP/test.csv")
val["path"] = "/content/drive/MyDrive/PROJET_DATA@354_NLP/clips/clips/"+val["ID"]+".mp3"
val.rename(columns = {'transcription':'sentence'}, inplace = True)
common_voice_val = Dataset.from_pandas(val)

In [73]:
common_voice_val = common_voice_val.remove_columns([ "ID"])

In [72]:
val.head()

,ID,path
0,d7182abeb9a53c4af01c54019e42525e8d709e7e040b68...,/content/drive/MyDrive/PROJET_DATA@354_NLP/cli...
1,938ef3a3fdd25efff8fb925ac3048736e5dd3d82881f10...,/content/drive/MyDrive/PROJET_DATA@354_NLP/cli...
2,0b65a85f018d7b9f7dba9f83bc87e2ed3105e8a7ed8536...,/content/drive/MyDrive/PROJET_DATA@354_NLP/cli...
3,f3b414026a67b04c3208e5230a09dc4d8c3a7ddec48f59...,/content/drive/MyDrive/PROJET_DATA@354_NLP/cli...
4,b98376a8de91dcd1fad26c6e5d8f3fbf5295413a8edc3e...,/content/drive/MyDrive/PROJET_DATA@354_NLP/cli...


In [74]:
common_voice_val = common_voice_val.map(speech_file_to_array_fn_test, remove_columns=common_voice_val.column_names)

Map:   0%|          | 0/1337 [00:00<?, ? examples/s]

In [75]:
common_voice_val = common_voice_val.map(prepare_dataset_test, remove_columns=common_voice_val.column_names, batch_size=8, num_proc=4, batched=True)

Map (num_proc=4):   0%|          | 0/1337 [00:00<?, ? examples/s]

In [76]:
common_voice_val.shape[0]

1337

In [77]:
common_voice_val.save_to_disk("/content/drive/MyDrive/PROJET_DATA@354_NLP/Pre_Data/Submission")

Saving the dataset (0/1 shards):   0%|          | 0/1337 [00:00<?, ? examples/s]

In [ ]:
common_voice_val = Dataset.load_from_disk("/content/drive/MyDrive/PROJET_DATA@354_NLP/Pre_Data/Submission")

In [78]:
final_pred = []
for i in tqdm(range(common_voice_val.shape[0])):    
    input_dict = processor(common_voice_val[i]["input_values"], return_tensors="pt", padding=True)

    logits = model(input_dict.input_values.to("cuda")).logits

    pred_ids = torch.argmax(logits, dim=-1)[0]
    prediction = processor.decode(pred_ids)
    final_pred.append(prediction)

  0%|          | 4/1337 [00:00<02:41,  8.27it/s]It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
  0%|          | 6/1337 [00:00<02:21,  9.40it/s]It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
  1%|          | 8/1337 [00:00<02:17,  9.68it/s]It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument 

In [79]:
val.columns

Index(['ID', 'path'], dtype='object')

In [80]:
val["transcription"] = final_pred
val["transcription"] = val["transcription"].str.capitalize()

In [81]:
val.head()

,ID,path,transcription
0,d7182abeb9a53c4af01c54019e42525e8d709e7e040b68...,/content/drive/MyDrive/PROJET_DATA@354_NLP/cli...,Groupe[unk]sanda[unk]
1,938ef3a3fdd25efff8fb925ac3048736e5dd3d82881f10...,/content/drive/MyDrive/PROJET_DATA@354_NLP/cli...,Djily[unk]creation[unk]
2,0b65a85f018d7b9f7dba9f83bc87e2ed3105e8a7ed8536...,/content/drive/MyDrive/PROJET_DATA@354_NLP/cli...,Rue[unk]a[unk]grand[unk]dakar[unk]
3,f3b414026a67b04c3208e5230a09dc4d8c3a7ddec48f59...,/content/drive/MyDrive/PROJET_DATA@354_NLP/cli...,Beénnally[unk]
4,b98376a8de91dcd1fad26c6e5d8f3fbf5295413a8edc3e...,/content/drive/MyDrive/PROJET_DATA@354_NLP/cli...,Brioche[unk]ée[unk]diamalaye[unk]


In [ ]:
final_pred2 = val["transcription"]
final_pred2 = [ch.replace("[unk]", " ") for ch in final_pred2]
val["transcription"] = final_pred2

In [17]:
val[["ID","transcription"]].head()

,ID,transcription
0,d7182abeb9a53c4af01c54019e42525e8d709e7e040b68...,Groupe sanda
1,938ef3a3fdd25efff8fb925ac3048736e5dd3d82881f10...,Djily creation
2,0b65a85f018d7b9f7dba9f83bc87e2ed3105e8a7ed8536...,Rue a grand dakar
3,f3b414026a67b04c3208e5230a09dc4d8c3a7ddec48f59...,Beénnally
4,b98376a8de91dcd1fad26c6e5d8f3fbf5295413a8edc3e...,Brioche ée diamalaye


In [ ]:
val[["ID","transcription"]].to_csv("/content/drive/MyDrive/PROJET_DATA@354_NLP/submission.csv",index=False)